In [5]:
import AdventOfCode

input <- readInt <$> dayString 3

In [73]:
-- The downward right diagonal goes 1, 9, 25, 49, ..., i.e map (^2) [1,3,5,...]
diagonals = takeWhile (<input) (map (^2) [1,3..])

-- The closest square before the input
square = last diagonals

squareRt = floor $ sqrt $ fromIntegral square

-- The difference between the next diagonal and the input
diff = (squareRt + 2)^2 - input

-- Manhattan distance is 2*length diagonals for diagonals, minus the difference
(length diagonals * 2)-diff

438

In [67]:
import Data.Function.Memoize

genDRDiagonal = memoize $ \n -> (^2) .head $ drop n [1,3..]
genURDiagonal n = let
    drDiagonal = genDRDiagonal n
    s = floor . sqrt $ fromIntegral drDiagonal
    in drDiagonal - ((s-1)*3)
genULDiagonal n = let
    drDiagonal = genDRDiagonal n
    s = floor . sqrt $ fromIntegral drDiagonal
    in drDiagonal - ((s-1)*2)
genDLDiagonal n = let
    drDiagonal = genDRDiagonal n
    s = floor . sqrt $ fromIntegral drDiagonal
    in drDiagonal - (s-1)
    
genPos (x,y) = case (x>=0, y>=0) of
    (True, True)  -> case cmp x y of
        LT -> genURDiagonal y + (y-x)
        EQ -> genURDiagonal x
        GT -> genURDiagonal x - (x-y)
    (True, False) -> case cmp x y of
        LT -> genDRDiagonal (abs y) - (abs y - x)
        EQ -> genDRDiagonal x
        GT -> genDRDiagonal (x-1) + (x - abs y)
    (False, True) -> case cmp x y of
        LT -> genULDiagonal (abs y) - (abs y - abs x)
        EQ -> genULDiagonal (abs x)
        GT -> genULDiagonal (abs x) + (abs x - abs y)
    (False, False) -> case cmp x y of
        LT -> genDLDiagonal (abs y) + (abs y - abs x)
        EQ -> genDLDiagonal (abs x)
        GT -> genDLDiagonal (abs x) - (abs x - abs y)
    where cmp x y = compare (abs x) (abs y)

In [130]:
getPos n = let
    diagonals = takeWhile (<n) $ map (^2) [1,3..]
    pos = length diagonals
    dr = genDRDiagonal pos
    dl = genDLDiagonal pos
    ur = genURDiagonal pos
    ul = genULDiagonal pos
    in case () of
    _ | n == dr           -> (pos, -pos)
      | n < dr && n >= dl -> (-pos + (n - dl), -pos)
      | n < dl && n >= ul -> (-pos, pos - (n - ul))
      | n < ul && n >= ur -> (pos - (n - ur), pos)
      | n < ur            -> (pos, pos - (ur - n))
    

-- Solve for difference in this way.
(x,y) = getPos input
abs x + abs y

438

In [131]:
import qualified Data.Map.Strict as M

-- nextPos

neighbours (x, y) =
    [ (x-1, y-1), (x, y-1), (x+1, y-1)
    , (x-1, y),             (x+1, y)
    , (x-1, y+1), (x, y+1), (x+1, y+1)
    ]

sumNeighbours g c@(x,y) = sum $ map (\c' -> M.findWithDefault 0 c' g) $ neighbours c

grid = M.fromList [((0,0), 1)]

loop gr i stop = let
    coord = getPos i
    value = sumNeighbours gr coord
    in if value > stop
        then (value, gr)
        else loop (M.insert coord value gr) (i+1) stop
        
fst $ loop grid 2 input

266330